# Welcome to pyLIMA tutorial! #

Let's simulate, and fit back :), some events.

# Alpha version, probably gonna change #

In [ ]:
## First import the required libraries

import numpy as np
import matplotlib.pyplot as plt
import os, sys
lib_path = os.path.abspath(os.path.join('../'))
sys.path.append(lib_path)

from pyLIMA import microlsimulator

In [ ]:
# Create your event, 

my_own_creation = microlsimulator.simulate_a_microlensing_event('A spectacular double source point lens', ra=270, dec=-30)

# Create some telescopes

# Name = Survey, altitude = 1000 m, longitude = -109.285399, latitude = -27.130814,filter = 'I',start_obs =2457000,
# end_obs = 2458000, sampling = 5, bad_weather_percentage = 10%, moon_windows_avoidance=6, minimum_alt=30)
my_survey = microlsimulator.simulate_a_telescope('survey', 0, 0, 0, 'I', 2457000, 2458000, 5,
                                 bad_weather_percentage=10.0 / 100, moon_windows_avoidance=6, minimum_alt=30)


# Name = 'SAAO_I', altitude = 400 m, longitude = 20.659279, latitude = -32.3959,filter = 'I',start_obs =2457689,
# end_obs = 2457730, sampling = 10, bad_weather_percentage = 50%, moon_windows_avoidance=2, minimum_alt=15)
my_own_telescope_1 = microlsimulator.simulate_a_telescope('SAAO_I', 400, 20.659279, -32.3959, 'I', 2457689, 2457730, 10,
                                        bad_weather_percentage=50.0 / 100, moon_windows_avoidance=2, minimum_alt=15)
# Name = 'SAAO_V', altitude = 400 m, longitude = 20.659279, latitude = -32.3959,filter = 'V',start_obs =2457689,
# end_obs = 2457730, sampling = 1, bad_weather_percentage = 50%, moon_windows_avoidance=10, minimum_alt=15)
my_own_telescope_2 = microlsimulator.simulate_a_telescope('SAAO_V', 400, 20.659279, -32.3959, 'V', 2457689, 2457730, 1,
                                        bad_weather_percentage=50.0 / 100, moon_windows_avoidance=10, minimum_alt=15)
# Add them to your event
my_own_creation.telescopes.append(my_survey)
my_own_creation.telescopes.append(my_own_telescope_1)
my_own_creation.telescopes.append(my_own_telescope_2)

# What model you want? Let's have DSPL!
my_own_model = microlsimulator.simulate_a_microlensing_model(my_own_creation, model='DSPL', parallax=['None', 0.0],
                                             xallarap=['None', 0.0],
                                             orbital_motion=['None', 0.0], source_spots='None')

# Find some model parameters
my_own_parameters = microlsimulator.simulate_microlensing_model_parameters(my_own_model)

# Which source magnitude? Which blending?

my_own_flux_parameters = microlsimulator.simulate_fluxes_parameters(my_own_creation.telescopes)
my_own_parameters += my_own_flux_parameters

# Transform into pyLIMA standards
pyLIMA_parameters = my_own_model.compute_pyLIMA_parameters(my_own_parameters)

# update the lightcurves in your event :
microlsimulator.simulate_lightcurve_flux(my_own_model, pyLIMA_parameters,  red_noise_apply='Yes')


#### Plot it!

for telescope in my_own_creation.telescopes:
    plt.errorbar(telescope.lightcurve_magnitude[:, 0], telescope.lightcurve_magnitude[:, 1],
                 yerr=telescope.lightcurve_magnitude[:, 2], fmt='.')

for key in my_own_model.model_dictionnary.keys():
    
    print key,my_own_parameters[my_own_model.model_dictionnary[key]]

plt.gca().invert_yaxis()
plt.show()

Let's try to fit this now!

In [ ]:
from pyLIMA import microlmodels

model_1 = microlmodels.create_model('PSPL', my_own_creation)
my_own_creation.fit(model_1,'DE')

model_2 = microlmodels.create_model('DSPL', my_own_creation)
my_own_creation.fit(model_2,'DE')

my_own_creation.fits[0].produce_outputs()
my_own_creation.fits[1].produce_outputs()

print my_own_creation.fits[0].model.model_type,'Chi2_LM :',my_own_creation.fits[0].outputs.fit_parameters.chichi
print my_own_creation.fits[1].model.model_type,'Chi2_LM :',my_own_creation.fits[1].outputs.fit_parameters.chichi

plt.show()